## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Broome,US,42.2506,-75.8330,17.62,69,40,10.36,scattered clouds
1,1,Kankon,IN,15.0167,74.0167,73.20,66,6,4.50,clear sky
2,2,Ushuaia,AR,-54.8000,-68.3000,38.86,81,40,26.46,scattered clouds
3,3,Carauari,BR,-4.8828,-66.8958,73.27,99,100,1.39,overcast clouds
4,4,Cape Town,ZA,-33.9258,18.4232,71.11,79,20,18.41,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Rikitea,PF,-23.1203,-134.9692,78.94,71,5,14.14,clear sky
18,18,Half Moon Bay,US,37.4636,-122.4286,78.98,51,0,3.44,clear sky
23,23,Katiola,CI,8.1333,-5.1000,77.63,67,6,3.02,clear sky
29,29,Arraial Do Cabo,BR,-22.9661,-42.0278,75.70,92,11,14.36,few clouds
31,31,Hithadhoo,MV,-0.6000,73.0833,82.06,75,38,12.80,scattered clouds
32,32,Mount Isa,AU,-20.7333,139.5000,87.57,45,100,17.27,overcast clouds
37,37,Gizo,SB,-8.1030,156.8419,84.42,68,98,6.67,overcast clouds
44,44,Atuona,PF,-9.8000,-139.0333,77.94,78,14,13.60,light rain
49,49,Puerto Ayora,EC,-0.7393,-90.3518,75.54,92,89,4.16,overcast clouds
51,51,San Policarpo,PH,12.1791,125.5072,80.83,73,100,12.84,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                186
City                   186
Country                186
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Current Description    186
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# - there were no empty rows

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Rikitea,PF,78.94,clear sky,-23.1203,-134.9692,
18,Half Moon Bay,US,78.98,clear sky,37.4636,-122.4286,
23,Katiola,CI,77.63,clear sky,8.1333,-5.1000,
29,Arraial Do Cabo,BR,75.70,few clouds,-22.9661,-42.0278,
31,Hithadhoo,MV,82.06,scattered clouds,-0.6000,73.0833,
32,Mount Isa,AU,87.57,overcast clouds,-20.7333,139.5000,
37,Gizo,SB,84.42,overcast clouds,-8.1030,156.8419,
44,Atuona,PF,77.94,light rain,-9.8000,-139.0333,
49,Puerto Ayora,EC,75.54,overcast clouds,-0.7393,-90.3518,
51,San Policarpo,PH,80.83,overcast clouds,12.1791,125.5072,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   186
Country                186
Max Temp               186
Current Description    186
Lat                    186
Lng                    186
Hotel Name             186
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))